<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [2]:
#открываем 
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv(r'C:\Users\Бегущий за пивом\Documents\шпаргалки\comments_project\toxic_comments.csv')
    

data

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data['toxic'].mean()

0.10161213369158527

Токсичных комментариев 10% от всех.Перед лемматизацией нужно все слова привести к нижнему регистру и отчистить от лишних символов

In [7]:
stop_words = nltk_stopwords.words('english')
type(stop_words)

list

In [8]:
def clear_text(text):
    
    #импортируем стоп-слова
    stop_words = nltk_stopwords.words('english')
    #в нижний регистр 
    text = text.lower()
    #заменяем все символы на пробелы 
    word_list = re.sub(r"[^a-z ]", ' ', text).split()
    sentence = [w for w in word_list if not w in stop_words]
    
    return ' '.join(sentence)

In [9]:
 data['clean_text'] = data['text'].apply(clear_text)
# data

Для правильной лемматизации используем аргумент lemmatize() указать правильный тег «part-of-speech» (POS-тег).В nltk для этого есть метод nltk.pos_tag(). Он принимает только список (список слов), даже если нужно передать только одно слово.nltk.pos_tag() возвращает кортеж с тегом POS. Ключевым моментом здесь является сопоставление POS-тегов NLTK с форматом, принятым лемматизатором wordnet.

In [10]:
# для учета контекста 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


#лемматизация 
def postag_lemm_text(text):
    
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemmatized_text


In [11]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Бегущий за пивом\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
data['lemm_text'] = data['clean_text'].apply(postag_lemm_text)

In [ ]:
# data.to_csv(r'D:\data_set\comm_lem.csv', index=False)

Очень долго  лемматизирует,поэтому сохранил. В дальнейшем будем просто экспортировать готовый фаил 

In [13]:
#data = pd.read_csv(r'D:\data_set\comm_lem.csv')
data.dropna(inplace=True)

**В этом разделе:**
* буквы переведены в нижний регистр, оставлена только латиница, удалены стоп-слова.
* комментарии лемматизированы с учётом части речи (POS-тегов).
* данные на всякий случай сохранены 

In [14]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
clean_text    0
lemm_text     0
dtype: int64

## Обучение

Разделим выборки

In [15]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, random_state=12345, shuffle=True)

corpus = data['lemm_text']

#для баланса классов указываем  stratify=data['toxic'].values
features_train, features_test, target_train, target_test = train_test_split(corpus, data['toxic'].values, 
    test_size=0.2, stratify=data['toxic'].values, shuffle=True, random_state=12345)


#делаем векторы 
tf_idf = TfidfVectorizer()
tf_idf_train = tf_idf.fit_transform(features_train)
tf_idf_test = tf_idf.transform(features_test)



Начнем с самой простой логистической регрессии.Посмотрим как долго и есть ли смысл искать гипперпараметры на моем ноутбуке.

#### Логистическая регрессия

In [16]:
log_reg = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=12345)


log_reg.f1 = cross_val_score(log_reg, tf_idf_train, target_train, cv=kfold, scoring='f1')


print('f1 для логистической регрессии: %.3f' %(log_reg.f1.mean()) )

f1 для логистической регрессии: 0.753


Пока нормально, теперь попробуем с LGBMClassifier 

#### LGBMClassifier 

In [18]:

LGBM = LGBMClassifier(n_estimators=180, class_weight='balanced', boosting_type='gbdt', 
                         objective='binary', random_state=12345)



LGBM.f1 = cross_val_score(LGBM, tf_idf_train, target_train, cv=kfold, scoring='f1')


print('f1 for LGBMClassifier: %.3f' %(LGBM.f1.mean()))

[LightGBM] [Info] Number of positive: 10316, number of negative: 91591
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.464442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 466799
[LightGBM] [Info] Number of data points in the train set: 101907, number of used features: 8648
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Number of positive: 10334, number of negative: 91573
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 4.784417 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 469130
[LightGBM] [Info] Number of data points in the train set: 101907, number of used features: 8697
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initsc

долговато как для 80 деревьев,даже использовать randomsearch не вижу смысла мое железо не потянет.Ну и лес на такое же количество деревьев 

#### RandomForestClassifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest  = RandomForestClassifier(n_estimators = 180 ,max_depth = 6, 
                                 class_weight='balanced', random_state=12345,n_jobs=-1)

forest.f1 = cross_val_score(forest, tf_idf_train, target_train, cv=kfold, scoring='f1')

print('f1 for RandomForest: %.3f' %(forest.f1.mean()))

Все очень плохо(

In [ ]:
table = pd.DataFrame([
    ['LogisticRegression',log_reg.f1.mean()],
    ['LGBMClassifier',LGBM.f1.mean()],
    ['RandomForestClassifier',forest.f1.mean()]
    ],
                     columns=['models','f1-score'])
table

,models,f1-score
0,LogisticRegression,0.752926
1,LGBMClassifier,0.748352
2,RandomForestClassifier,0.343406


### Тест 

In [ ]:
log_reg.fit(tf_idf_train, target_train)

prediction= log_reg.predict(tf_idf_test)
test_f1 = f1_score(target_test, prediction)
print(test_f1)

0.7538461538461538


На тесте f1-score = 0.753, преодолел порог.

## Выводы

* Предобработали данные:
  1. Отчистили комментарии от ненужных символов 
  2. произвели лемматизированы с учётом части речи (POS-тегов)
  3. Разбили на тестовую и обучающую выборки 
* Произвели обучение трех моделей с использованием кросс-валидации:
  1. LogisticRegression	0.752926
  2. LGBMClassifier	0.748352
  3. RandomForestClassifier	0.343406
* На тетовой выборке LogisticRegression перешла порог в 0,75 

Возможно можно было достичь лучших результатов поискав гиперпараметры, LGBMClassifier на 180 деревьях учился 4 минуты,лемматизация производилась около 15,так что проект упирается в технические отграничения железа.Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Для нашей задачи хватило и логистической регрессии. 